In [ ]:
# %%capture
# %pip install roiextractors
# %pip install neuroconv==0.2.4
# %pip install tifffile
# %pip install opencv-python-headless

In [1]:
from microns_phase3 import nda
from pathlib import Path
import os
from microns_to_nwb.convert_session import convert_session, parallel_convert_sessions

Connecting microns@db.datajoint.com:3306


Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`
/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`


In [2]:
# Source data file paths
scan_keys = list(nda.Scan.proj())

# The list of file paths to the downloaded imaging data, one per scan
ophys_file_version = 2
ophys_file_directory = f"/mnt/at-export01/17797-export/two_photon_functional_scans/v{ophys_file_version}/"
ophys_file_paths = [Path(ophys_file_directory + '_'.join(['functional','scan','17797',
                                                         *[str(s) for s in scan_key.values()],
                                                         'v'+str(ophys_file_version)+'.tif'])) 
                    for scan_key in scan_keys]

# The list of file paths to the downloaded stimulus movie files, one per scan
movie_file_version = 4
movie_file_directory = f"/mnt/at-export01/17797-export/stimulus_movies/v{movie_file_version}/"
movie_file_paths = [Path(movie_file_directory + '_'.join(['stimulus','17797',
                                                         *[str(s) for s in scan_key.values()],
                                                         'v'+str(movie_file_version)+'.avi'])) 
                    for scan_key in scan_keys]

# target file path where NWB files will be written
nwb_output_path = Path("/mnt/at-export01/17797-export/DANDI_NWB/")
nwbfile_list = [nwb_output_path / ophys_file_path.stem / f"{ophys_file_path.stem}.nwb" for ophys_file_path in ophys_file_paths]

# create subdirectories for NWB files
nwbfile_folder_paths = [nwb_output_path / ophys_file_path.stem for ophys_file_path in ophys_file_paths]
_ = [os.makedirs(nwbfile_folder_path, exist_ok=True) for nwbfile_folder_path in nwbfile_folder_paths]

In [3]:
%pip show neuroconv

Name: neuroconv
Version: 0.2.4
Summary: Convert data from proprietary formats to NWB format.
Home-page: https://github.com/catalystneuro/neuroconv
Author: Cody Baker, Szonja Weigl, Heberto Mayorquin, Luiz Tauffer, and Ben Dichter.
Author-email: ben.dichter@catalystneuro.com
License: UNKNOWN
Location: /usr/local/lib/python3.8/site-packages
Requires: dandi, h5py, hdmf, jsonschema, numpy, pandas, psutil, pynwb, PyYAML, scipy, tqdm
Required-by: microns-to-nwb
Note: you may need to restart the kernel to use updated packages.


In [4]:
convert_session(nwbfile_path = nwbfile_list[0],
                ophys_file_path = str(ophys_file_paths[0]),
                stimulus_movie_file_path = str(movie_file_paths[0]),
                dandiset_id = None,
                verbose=True)

/mnt/lab/users/Paul/git_repos/MICrONS-to-nwb/src/microns_to_nwb/convert_session.py:60: UserWarning: Writing behavior data to NWB with negative timestamps is not recommended,times are shifted to the earliest behavioral timestamp by 10.14944010375001 seconds.
  warn(
/usr/local/lib/python3.8/site-packages/pynwb/file.py:471: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


Behavior, trials, and Fluorescence traces are added from datajoint.
Source data is valid!
Metadata is valid!
conversion_options is valid!


/mnt/lab/users/Paul/git_repos/MICrONS-to-nwb/src/microns_to_nwb/convert_session.py:127: UserWarning: There was an error during conversion. The source files are not removed. The full traceback: 'nwbfile_path' exists at location, 'overwrite' is False (append mode), but an in-memory 'nwbfile' object was passed! Cannot reconcile which nwbfile object to write.
  warn(f"There was an error during conversion. The source files are not removed. The full traceback: {e}")


Cleaning up ...


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/at-export01/17797-export/two_photon_functional_scans/v2/functional_scan_17797_4_7_v2.tif'

In [6]:
# parallel_convert_sessions(
#     num_parallel_jobs = len(ophys_file_paths),
#     nwbfile_list = nwbfile_list,
#     ophys_file_paths = ophys_file_paths,
#     stimulus_movie_file_paths = movie_file_paths,
#     dandiset_id = None,
#     )